References  
https://velog.io/@songjeongwoo/series/%EB%8D%B0%EC%9D%B4%ED%84%B0-%EC%A0%84%EC%B2%98%EB%A6%AC   
https://zhining.tistory.com/29  
  
**순서 : 가변수화 -> 스케일링 -> 결측치 kNNImputer**   
  
Logistic Regression :  스케일링 o, 결측치 처리 o  
Decision Tree: 스케일링 x, 결측치 처리 o   
  
학습셋과 테스트셋은 반드시 동일한 전처리 과정을 거쳐야함

# Preprocessing

In [2]:
%matplotlib inline
import os
from pathlib import Path
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix, parallel_coordinates
import seaborn as sns
import matplotlib.pylab as plt
import warnings

In [3]:
warnings.filterwarnings('ignore')

In [4]:
train = pd.read_csv("C:/Users/kimch/Desktop/9565_hospital_data/train.csv")
test = pd.read_csv("C:/Users/kimch/Desktop/9565_hospital_data/test.csv")

In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [6]:
# 왜인지 모르겠는데 (-> NaN값 있어서? )openDate가 float로 나와서 int로 형변환해줌 
test["openDate"]

0      19981125.0
1      20160309.0
2      19890427.0
3      20100226.0
4      20040604.0
5      20081224.0
6      20020513.0
7      20070110.0
8      20151209.0
9      19851223.0
10     20071109.0
11     20080404.0
12     19820630.0
13     20080801.0
14     20030102.0
15     19950330.0
16     20080901.0
17     20070817.0
18     20000308.0
19     20130208.0
20     19970228.0
21     19990708.0
22     20060629.0
23     20160408.0
24     20000901.0
25     20051207.0
26     19940411.0
27     20140912.0
28     20061017.0
29     19760409.0
30     19971230.0
31     20080407.0
32     20001227.0
33     19880803.0
34     20070115.0
35     20121030.0
36     20110901.0
37     20070808.0
38     20091014.0
39     19880524.0
40     20081210.0
41     20060724.0
42     20130422.0
43     19930101.0
44     19820520.0
45     20110901.0
46     20090218.0
47     20141106.0
48     20111006.0
49     20081226.0
50     19830822.0
51     20060929.0
52     20050321.0
53     20081223.0
54     19990504.0
55     200

In [7]:
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 20)

In [8]:
test = test.replace({"openDate":np.NaN}, {"openDate":0})

In [9]:
test = test.astype({'openDate':"int"})

In [10]:
# 이러면 또 float형으로 바뀜
# test = test.replace({"openDate":0}, {"openDate":np.NaN})

In [11]:
train.rename(columns={'Profit1':"profit1", "Debt2":"debt2"}, inplace=True)
test.rename(columns={'Profit1':"profit1", "Debt2":"debt2"}, inplace=True)

In [12]:
# bedCount 
bins = [0, 30, 100, 300, train["bedCount"].max()]
# 1: 30이하, 2: 30초과 100 이하, 3: 100초과 300이하, 4: 300초과 
labels = [1, 2, 3, 4]
train["bedCount_class"] = pd.cut(train["bedCount"], bins, labels=labels)
test["bedCount_class"] = pd.cut(test["bedCount"], bins, labels=labels)

In [13]:
train = train.drop(columns=["bedCount"])
test = test.drop(columns=["bedCount"])

## 범주형 변수 One-hot Encoding

#### sido

In [14]:
train = train.replace({"sido":("choongbuk","choongnam", "sejong", "daejeon")}, "choongchung")
train = train.replace({"sido":("gyeongbuk", "gyeongnam", "ulsan", "busan", "daegu")}, "gyeongsang")
train = train.replace({"sido":("jeonbuk", "jeonnam", "gwangju")}, "jeonra")
train = train.replace({"sido":("gyeonggi", "incheon")}, "gyeongin")

In [15]:
test = test.replace({"sido":("choongbuk","choongnam", "sejong", "daejeon")}, "choongchung")
test = test.replace({"sido":("gyeongbuk", "gyeongnam", "ulsan", "busan", "daegu")}, "gyeongsang")
test = test.replace({"sido":("jeonbuk", "jeonnam", "gwangju")}, "jeonra")
test = test.replace({"sido":("gyeonggi", "incheon")}, "gyeongin")

#### instkind

In [16]:
train["instkind"].value_counts()

nursing_hospital        144
hospital                 88
general_hospital         37
clinic                   18
traditional_hospital      8
traditional_clinic        4
dental_clinic             1
Name: instkind, dtype: int64

In [17]:
# test에는 dental clinic이 없음 -> train에도 dental_clinic 하나이므로 dental_clinic은 clinic에 넣어주기로
test["instkind"].value_counts()

nursing_hospital        63
hospital                36
general_hospital        19
clinic                   4
traditional_clinic       2
traditional_hospital     1
Name: instkind, dtype: int64

In [18]:
train = train.replace({"instkind":("dental_clinic")}, "clinic")

In [19]:
train["instkind"].value_counts()

nursing_hospital        144
hospital                 88
general_hospital         37
clinic                   19
traditional_hospital      8
traditional_clinic        4
Name: instkind, dtype: int64

#### sgg

In [20]:
train = train.drop(columns=["sgg"])
test = test.drop(columns=["sgg"])

#### ownerChange

In [21]:
train.loc[train["ownerChange"]=="same","ownerChange"] = 0
train.loc[train["ownerChange"]=="change","ownerChange"] = 1 

In [22]:
test.loc[test["ownerChange"]=="same","ownerChange"] = 0
test.loc[test["ownerChange"]=="change","ownerChange"] = 1 

#### OC

In [23]:
# test는 OC NaN임
train.loc[train["OC"]==" close", "OC"] = 0
train.loc[train["OC"]=="open", "OC"] = 1

#### one-hot encoding : sido, instkind

In [24]:
from sklearn.preprocessing import OneHotEncoder

In [25]:
ohe = OneHotEncoder(sparse=False)
ohe_sido = ohe.fit_transform(train[["sido"]])
ohe_sido

array([[1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.]])

In [26]:
ohe.categories_[0]

array(['choongchung', 'gangwon', 'gyeongin', 'gyeongsang', 'jeonra',
       'seoul'], dtype=object)

In [27]:
pd.DataFrame(ohe_sido, columns=["sido_" + col for col in ohe.categories_[0]])

,sido_choongchung,sido_gangwon,sido_gyeongin,sido_gyeongsang,sido_jeonra,sido_seoul
0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...
296,0.0,0.0,0.0,0.0,1.0,0.0
297,1.0,0.0,0.0,0.0,0.0,0.0
298,0.0,0.0,0.0,1.0,0.0,0.0
299,0.0,0.0,1.0,0.0,0.0,0.0


In [28]:
train = pd.concat([train.drop(columns=["sido"]), 
                   pd.DataFrame(ohe_sido, columns=["sido_" + col for col in ohe.categories_[0]])], axis=1)
train

,inst_id,OC,openDate,instkind,revenue1,salescost1,sga1,salary1,noi1,noe1,...,employee1,employee2,ownerChange,bedCount_class,sido_choongchung,sido_gangwon,sido_gyeongin,sido_gyeongsang,sido_jeonra,sido_seoul
0,1,1,20071228,nursing_hospital,4.217530e+09,0.000000e+00,3.961135e+09,2.033835e+09,15652441.0,1.523624e+07,...,62.0,64.0,0,3,1.0,0.0,0.0,0.0,0.0,0.0
1,3,1,19970401,general_hospital,NaN,NaN,NaN,NaN,NaN,NaN,...,801.0,813.0,0,4,0.0,0.0,0.0,1.0,0.0,0.0
2,4,1,20161228,nursing_hospital,1.004522e+09,5.154837e+08,4.472197e+08,2.964023e+08,76156.0,3.000000e+04,...,234.0,1.0,0,4,0.0,0.0,1.0,0.0,0.0,0.0
3,7,1,20000814,general_hospital,7.250734e+10,0.000000e+00,7.067740e+10,3.178605e+10,506223059.0,1.259568e+09,...,663.0,663.0,0,4,0.0,0.0,1.0,0.0,0.0,0.0
4,9,1,20050901,general_hospital,4.904354e+10,0.000000e+00,4.765605e+10,2.446078e+10,112352259.0,1.419089e+09,...,206.0,197.0,0,3,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,422,1,20050211,nursing_hospital,7.614697e+09,4.153475e+08,5.903119e+09,3.467861e+09,42098974.0,7.439421e+08,...,193.0,141.0,0,3,0.0,0.0,0.0,0.0,1.0,0.0
297,423,0,20140618,hospital,6.717144e+09,8.200000e+09,0.000000e+00,0.000000e+00,186359.0,1.398513e+08,...,79.0,79.0,0,NaN,1.0,0.0,0.0,0.0,0.0,0.0
298,426,0,19830705,hospital,0.000000e+00,0.000000e+00,5.479428e+07,7.500000e+06,121022160.0,1.379810e+08,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,0.0
299,427,0,20070101,nursing_hospital,2.116892e+09,2.681748e+08,3.286245e+09,2.231944e+09,123665065.0,5.366919e+08,...,100.0,NaN,NaN,3,0.0,0.0,1.0,0.0,0.0,0.0


In [29]:
ohe_instkind = ohe.fit_transform(train[["instkind"]])
ohe_instkind

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [30]:
ohe.categories_[0]

array(['clinic', 'general_hospital', 'hospital', 'nursing_hospital',
       'traditional_clinic', 'traditional_hospital', nan], dtype=object)

In [31]:
oheinstkind_df = pd.DataFrame(ohe_instkind, columns=["instkind_clinic", "instkind_general_hospital", "instkind_hospital", "instkind_nursing_hospital", "instkind_traditional_clinic", "instkind_traditional_hospital", "instkind_nan"])
oheinstkind_df = oheinstkind_df.iloc[:,:7]
oheinstkind_df

,instkind_clinic,instkind_general_hospital,instkind_hospital,instkind_nursing_hospital,instkind_traditional_clinic,instkind_traditional_hospital,instkind_nan
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
296,0.0,0.0,0.0,1.0,0.0,0.0,0.0
297,0.0,0.0,1.0,0.0,0.0,0.0,0.0
298,0.0,0.0,1.0,0.0,0.0,0.0,0.0
299,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [32]:
train = pd.concat([train.drop(columns=["instkind"]), oheinstkind_df], axis=1)
train

,inst_id,OC,openDate,revenue1,salescost1,sga1,salary1,noi1,noe1,interest1,...,sido_gyeongsang,sido_jeonra,sido_seoul,instkind_clinic,instkind_general_hospital,instkind_hospital,instkind_nursing_hospital,instkind_traditional_clinic,instkind_traditional_hospital,instkind_nan
0,1,1,20071228,4.217530e+09,0.000000e+00,3.961135e+09,2.033835e+09,15652441.0,1.523624e+07,1.323624e+07,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,3,1,19970401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,4,1,20161228,1.004522e+09,5.154837e+08,4.472197e+08,2.964023e+08,76156.0,3.000000e+04,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,7,1,20000814,7.250734e+10,0.000000e+00,7.067740e+10,3.178605e+10,506223059.0,1.259568e+09,1.196881e+09,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,9,1,20050901,4.904354e+10,0.000000e+00,4.765605e+10,2.446078e+10,112352259.0,1.419089e+09,1.307249e+09,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,422,1,20050211,7.614697e+09,4.153475e+08,5.903119e+09,3.467861e+09,42098974.0,7.439421e+08,1.875761e+08,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
297,423,0,20140618,6.717144e+09,8.200000e+09,0.000000e+00,0.000000e+00,186359.0,1.398513e+08,1.296975e+08,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
298,426,0,19830705,0.000000e+00,0.000000e+00,5.479428e+07,7.500000e+06,121022160.0,1.379810e+08,1.379810e+08,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
299,427,0,20070101,2.116892e+09,2.681748e+08,3.286245e+09,2.231944e+09,123665065.0,5.366919e+08,5.143688e+08,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [33]:
ohe_sido_t = ohe.fit_transform(test[["sido"]])
ohesido_df_t = pd.DataFrame(ohe_sido_t, columns=["sido_" + col for col in ohe.categories_[0]])
test = pd.concat([test.drop(columns=["sido"]), ohesido_df_t], axis=1)

In [34]:
ohe_instkind = ohe.fit_transform(test[["instkind"]])
oheinstkind_df = pd.DataFrame(ohe_instkind, columns=["instkind_clinic", "instkind_general_hospital", "instkind_hospital", "instkind_nursing_hospital", "instkind_traditional_clinic", "instkind_traditional_hospital", "instkind_nan"])
oheinstkind_df = oheinstkind_df.iloc[:,:7]  # nan 제외
test = pd.concat([test.drop(columns=["instkind"]), oheinstkind_df], axis=1)

In [35]:
train

,inst_id,OC,openDate,revenue1,salescost1,sga1,salary1,noi1,noe1,interest1,...,sido_gyeongsang,sido_jeonra,sido_seoul,instkind_clinic,instkind_general_hospital,instkind_hospital,instkind_nursing_hospital,instkind_traditional_clinic,instkind_traditional_hospital,instkind_nan
0,1,1,20071228,4.217530e+09,0.000000e+00,3.961135e+09,2.033835e+09,15652441.0,1.523624e+07,1.323624e+07,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,3,1,19970401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,4,1,20161228,1.004522e+09,5.154837e+08,4.472197e+08,2.964023e+08,76156.0,3.000000e+04,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,7,1,20000814,7.250734e+10,0.000000e+00,7.067740e+10,3.178605e+10,506223059.0,1.259568e+09,1.196881e+09,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,9,1,20050901,4.904354e+10,0.000000e+00,4.765605e+10,2.446078e+10,112352259.0,1.419089e+09,1.307249e+09,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,422,1,20050211,7.614697e+09,4.153475e+08,5.903119e+09,3.467861e+09,42098974.0,7.439421e+08,1.875761e+08,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
297,423,0,20140618,6.717144e+09,8.200000e+09,0.000000e+00,0.000000e+00,186359.0,1.398513e+08,1.296975e+08,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
298,426,0,19830705,0.000000e+00,0.000000e+00,5.479428e+07,7.500000e+06,121022160.0,1.379810e+08,1.379810e+08,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
299,427,0,20070101,2.116892e+09,2.681748e+08,3.286245e+09,2.231944e+09,123665065.0,5.366919e+08,5.143688e+08,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [36]:
test

,inst_id,OC,openDate,revenue1,salescost1,sga1,salary1,noi1,noe1,interest1,...,sido_jeju,sido_jeonra,sido_seoul,instkind_clinic,instkind_general_hospital,instkind_hospital,instkind_nursing_hospital,instkind_traditional_clinic,instkind_traditional_hospital,instkind_nan
0,2,NaN,19981125,6.682486e+10,0.000000e+00,6.565709e+10,3.223695e+10,4.314093e+09,4.901517e+09,1.775872e+09,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,5,NaN,20160309,3.495758e+10,0.000000e+00,3.259270e+10,1.672254e+10,6.871024e+07,1.981033e+09,1.936455e+09,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,6,NaN,19890427,2.326031e+10,2.542571e+09,2.308749e+10,1.168734e+10,1.601943e+09,1.380941e+09,8.101204e+08,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,8,NaN,20100226,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.850652e+09,2.060989e+09,0.000000e+00,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,10,NaN,20040604,5.037025e+10,0.000000e+00,4.855803e+10,2.296346e+10,1.534452e+09,0.000000e+00,2.573804e+09,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,424,NaN,20011012,2.233031e+10,8.484657e+08,1.849255e+10,1.232241e+10,4.236289e+08,1.304817e+09,4.179491e+08,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
123,425,NaN,20010530,1.833906e+10,0.000000e+00,1.760117e+10,6.824241e+09,3.145447e+07,2.122632e+07,2.095582e+07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
124,429,NaN,20010101,1.424266e+10,1.726174e+09,1.148722e+10,6.279155e+09,2.723450e+07,8.607279e+08,8.471681e+08,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
125,430,NaN,20010201,4.892710e+10,4.157148e+10,4.721485e+09,1.514547e+09,8.509815e+07,1.270081e+09,8.379266e+08,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## 수치형 변수 : Log-Scaling

References: https://velog.io/@hhhs101/scaling

In [37]:
numerical_col = ['revenue1', 'salescost1', 'sga1', 'salary1', 'noi1', 'noe1', 'interest1', 'ctax1', 'profit1', 'liquidAsset1', 
                 'quickAsset1', 'receivableS1', 'inventoryAsset1', 'nonCAsset1', 'tanAsset1', 'OnonCAsset1', 'receivableL1', 'debt1', 'liquidLiabilities1', 'shortLoan1',
                 'NCLiabilities1', 'longLoan1', 'netAsset1', 'surplus1', 'revenue2', 'salescost2', 'sga2', 'salary2', 'noi2', 'noe2',
                 'interest2', 'ctax2', 'profit2', 'liquidAsset2', 'quickAsset2', 'receivableS2', 'inventoryAsset2', 'nonCAsset2', 'tanAsset2', 'OnonCAsset2',
                 'receivableL2', 'debt2', 'liquidLiabilities2', 'shortLoan2', 'NCLiabilities2', 'longLoan2','netAsset2', 'surplus2', 'employee1', 'employee2']

In [38]:
# employee1, employee2 천단위 넘어가면 , 가 있어서 숫자형이 아닌 object로 들어가 있음 

In [39]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [40]:
# train, test employee1, employee2 1, 들어가는 관측치 다 , 빼주기 위해 확인 
#train["employee1"].value_counts()
#train["employee2"].value_counts()
#test["employee1"].value_counts()
#test["employee2"].value_counts()

In [41]:
train = train.replace({"employee1": "1,816"}, 1816)
train = train.replace({"employee1": "1,637"}, 1637)
train = train.replace({"employee1": "1,065"}, 1065)
train = train.replace({"employee2": "1,104"}, 1104)
train = train.replace({"employee2": "1,547"}, 1547)
train = train.replace({"employee2": "1,065"}, 1065)

In [42]:
test = test.replace({"employee1": "1,816"}, 1816)
test = test.replace({"employee1": "1,637"}, 1637)
test = test.replace({"employee1": "1,065"}, 1065)
test = test.replace({"employee2": "1,104"}, 1104)
test = test.replace({"employee2": "1,547"}, 1547)
test = test.replace({"employee2": "1,065"}, 1065)

In [43]:
# 왜도 확인: 개별의 feature, 또는 target의 분포도가 skew가 심하면 log 변환
# 왜도 그래프로 확인 : sns.distplot(df['col'])

In [44]:
# 모든 수치형 변수가 왜도가 1보다 크므로 로그변환
print("skewness")
for col in numerical_col:
    print("%s : %.2f" %(col, train[col].skew()))

skewness
revenue1 : 3.76
salescost1 : 8.89
sga1 : 3.12
salary1 : 3.55
noi1 : 7.37
noe1 : 5.03
interest1 : 4.69
ctax1 : 6.65
profit1 : 2.04
liquidAsset1 : 3.80
quickAsset1 : 3.79
receivableS1 : 4.96
inventoryAsset1 : 4.43
nonCAsset1 : 4.30
tanAsset1 : 4.54
OnonCAsset1 : 5.71
receivableL1 : 17.12
debt1 : 3.07
liquidLiabilities1 : 2.68
shortLoan1 : 2.88
NCLiabilities1 : 3.76
longLoan1 : 4.20
netAsset1 : 6.75
surplus1 : 14.57
revenue2 : 3.68
salescost2 : 8.47
sga2 : 3.08
salary2 : 3.54
noi2 : 4.19
noe2 : 4.10
interest2 : 4.27
ctax2 : 7.21
profit2 : 3.57
liquidAsset2 : 3.94
quickAsset2 : 3.95
receivableS2 : 6.78
inventoryAsset2 : 4.29
nonCAsset2 : 4.68
tanAsset2 : 5.07
OnonCAsset2 : 5.21
receivableL2 : 17.12
debt2 : 2.95
liquidLiabilities2 : 4.42
shortLoan2 : 3.03
NCLiabilities2 : 3.39
longLoan2 : 3.15
netAsset2 : 6.39
surplus2 : 11.14
employee1 : 2.97
employee2 : 3.20


In [45]:
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 20)

In [46]:
# test.info()를 확인해봤을 때 train은 안그런데 openDate가 float 형태, employee1, employee2가 object 형태여서 형변환해줌 
# openDate도 test에서만 float 형태,,, 일단 NaN있어서 형변환하려하며 에러떠서 안바꿔줌
test["employee1"] = pd.to_numeric(test["employee1"])
test["employee2"] = pd.to_numeric(test["employee2"])

In [47]:
for col in numerical_col:
    train[col] = np.log1p(train[col])

In [48]:
train

,inst_id,OC,openDate,revenue1,salescost1,sga1,salary1,noi1,noe1,interest1,...,sido_gyeongsang,sido_jeonra,sido_seoul,instkind_clinic,instkind_general_hospital,instkind_hospital,instkind_nursing_hospital,instkind_traditional_clinic,instkind_traditional_hospital,instkind_nan
0,1,1,20071228,22.162515,0.000000,22.099796,21.433189,16.566138,16.539187,16.398469,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,3,1,19970401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,4,1,20161228,20.727778,20.060616,19.918561,19.507228,11.240552,10.308986,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,7,1,20000814,25.006954,0.000000,24.981392,24.182293,20.042488,20.954034,20.902985,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,9,1,20050901,24.615974,0.000000,24.587275,23.920337,18.537150,21.073281,20.991191,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,422,1,20050211,22.753346,19.844626,22.498747,21.966804,17.555534,20.427474,19.049695,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
297,423,0,20140618,22.627929,22.827400,0.000000,0.000000,12.135436,18.756090,18.680716,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
298,426,0,19830705,0.000000,0.000000,17.819096,15.830414,18.611484,18.742626,18.742626,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
299,427,0,20070101,21.473215,19.407150,21.913011,21.526139,18.633087,20.100935,20.058451,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [49]:
for col in numerical_col:
    test[col] = np.log1p(test[col])

In [50]:
test

,inst_id,OC,openDate,revenue1,salescost1,sga1,salary1,noi1,noe1,interest1,...,sido_jeju,sido_jeonra,sido_seoul,instkind_clinic,instkind_general_hospital,instkind_hospital,instkind_nursing_hospital,instkind_traditional_clinic,instkind_traditional_hospital,instkind_nan
0,2,NaN,19981125,24.925341,0.000000,24.907711,24.196379,22.185153,22.312810,21.297558,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,5,NaN,20160309,24.277401,0.000000,24.207354,23.540024,18.045409,21.406884,21.384125,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,6,NaN,19890427,23.870014,21.656442,23.862557,23.181772,21.194483,21.046031,20.512693,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,8,NaN,20100226,0.000000,0.000000,0.000000,0.000000,22.302379,21.446452,0.000000,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,10,NaN,20040604,24.642667,0.000000,24.606025,23.857170,21.151439,0.000000,21.668651,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,424,NaN,20011012,23.829211,20.558940,23.640634,23.234685,19.864368,20.989329,19.850870,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
123,425,NaN,20010530,23.632299,0.000000,23.591231,22.643747,17.264052,16.870753,16.857927,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
124,429,NaN,20010101,23.379508,21.269173,23.164501,22.560501,17.119995,20.573289,20.557410,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
125,430,NaN,20010201,24.613597,24.450680,22.275389,21.138382,18.259316,20.962347,20.546441,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## 결측치 처리 kNNImputer

In [51]:
# 결측치 0 -> nan 하려했는데 원핫인코딩을 하니깐 0 이 너무 많이 생겨서 안함 -> 0도 어쨌든 자체적으로 분석 할 것..? 
# Train: 결측치, 스케일링 완료된 train
# Test: 결측치, 스케일링 완료된 test

In [52]:
from sklearn.impute import KNNImputer

In [53]:
imputer = KNNImputer(n_neighbors=5)

In [54]:
# train
missingValues = pd.DataFrame(train.isnull().sum(), columns=["count"])
missingValues = missingValues[missingValues["count"] > 0]

In [55]:
Nalst = missingValues.index.tolist()

In [56]:
train_exceptNa = train.drop(columns=Nalst)
train_filled = imputer.fit_transform(train[Nalst])
train_filled_df = pd.DataFrame(train_filled, columns=Nalst)

In [57]:
Train = pd.concat([train_exceptNa, train_filled_df], axis=1)

In [58]:
# test
missingValues = pd.DataFrame(test.isnull().sum(), columns=["count"])
missingValues = missingValues[missingValues["count"] > 0]

In [59]:
Nalst = missingValues.index.tolist()
Nalst = Nalst[1:]

In [60]:
test_exceptNa = test.drop(columns=Nalst)
test_filled = imputer.fit_transform(test[Nalst])
test_filled_df = pd.DataFrame(test_filled, columns=Nalst)

In [61]:
Test = pd.concat([test_exceptNa, test_filled_df], axis=1)

## 그 외 처리들

In [62]:
# inst_id를 index로
Train = Train.set_index("inst_id")
# openDate를 DatetimeIndex로: train은 되는데 test는 안됨,,, 왜인지는 모르겠는데 unconverted라고
# Train["openDate"] = pd.to_datetime(Train.openDate, format="%Y%m%d")
Train = Train.drop(columns=["openDate"])

In [63]:
Test = Test.set_index("inst_id")
# Test["openDate"] = pd.to_datetime(Test.openDate, format="%Y%m%d")
Test = Test.drop(columns=["openDate"])

**순서: dataset split -> feature selection -> over/under sampling (train set)**

# Train -> Train, Valid

In [63]:
from sklearn.model_selection import train_test_split

In [64]:
# Logistic Regression을 위한 데이터셋이었는데 Decision Tree에서도 쓰기로 함 
X = Train.drop(columns=["OC"])
y = Train["OC"]

train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

In [65]:
# float형이면 LogisticRegression 돌릴 때 에러 뜸
train_y = train_y.astype('int')
valid_y = valid_y.astype('int')

# Feature Selection

In [66]:
from sklearn.feature_selection import SelectFromModel

In [67]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression

In [68]:
embeded_lr_selector = SelectFromModel(LogisticRegression(penalty="l2"))
embeded_lr_selector.fit(train_X, train_y)

SelectFromModel(estimator=LogisticRegression())

In [69]:
embeded_lr_support = embeded_lr_selector.get_support()
embeded_lr_feature = train_X.loc[:,embeded_lr_support].columns.tolist()
print(str(len(embeded_lr_feature)), "selected features")

27 selected features


In [70]:
# Decision Tree 
from sklearn.tree import DecisionTreeClassifier

In [71]:
embeded_dt_selector = SelectFromModel(DecisionTreeClassifier(random_state=0))
embeded_dt_selector.fit(train_X, train_y)

SelectFromModel(estimator=DecisionTreeClassifier(random_state=0))

In [72]:
embeded_dt_support = embeded_dt_selector.get_support()
embeded_dt_feature = train_X.loc[:,embeded_dt_support].columns.tolist()
print(str(len(embeded_dt_feature)), "selected features")

13 selected features


In [73]:
features = train_X.columns.tolist()

In [74]:
# Summary 
pd.set_option("display.max_rows", 40)

feature_selection_df = pd.DataFrame({'Feature': features, "Logistic": embeded_lr_support, "DecisionTree": embeded_dt_support})
feature_selection_df["Total"] = np.sum(feature_selection_df, axis=1)
feature_selection_df = feature_selection_df.sort_values(["Total", "Feature"], ascending=False)
feature_selection_df.head(40)

,Feature,Logistic,DecisionTree,Total
15,sga1,True,True,2
16,salary1,True,True,2
13,revenue1,True,True,2
49,inventoryAsset2,True,True,2
60,surplus2,True,False,1
36,surplus1,True,False,1
4,sido_jeonra,True,False,1
3,sido_gyeongsang,True,False,1
0,sido_choongchung,True,False,1
56,shortLoan2,True,False,1


In [75]:
pd.set_option("display.max_rows", 20)

# OverSampling

In [76]:
# oversample only train set

In [77]:
train_y.value_counts()

1    228
0     12
Name: OC, dtype: int64

In [78]:
from imblearn.over_sampling import SMOTE

In [79]:
smote = SMOTE(random_state=42)

train_X_smote, train_y_smote = smote.fit_resample(train_X, train_y)

In [80]:
train_y_smote.value_counts()

1    228
0    228
Name: OC, dtype: int64

# Fitting Model

## Logistic Regression

In [81]:
embeded_lr_feature

['sido_choongchung',
 'sido_gyeongsang',
 'sido_jeonra',
 'instkind_nursing_hospital',
 'instkind_traditional_hospital',
 'revenue1',
 'sga1',
 'salary1',
 'noe1',
 'interest1',
 'profit1',
 'nonCAsset1',
 'OnonCAsset1',
 'debt1',
 'surplus1',
 'noi2',
 'noe2',
 'interest2',
 'profit2',
 'inventoryAsset2',
 'OnonCAsset2',
 'shortLoan2',
 'NCLiabilities2',
 'longLoan2',
 'surplus2',
 'ownerChange',
 'bedCount_class']

In [82]:
logit_reg = LogisticRegression(penalty="l2", C=1e42, solver="liblinear")
logit_reg.fit(train_X_smote[embeded_lr_feature], train_y_smote)

LogisticRegression(C=1e+42, solver='liblinear')

In [83]:
from dmba import classificationSummary

In [84]:
classificationSummary(train_y_smote, logit_reg.predict(train_X_smote[embeded_lr_feature]))

Confusion Matrix (Accuracy 0.9978)

       Prediction
Actual   0   1
     0 228   0
     1   1 227


In [85]:
classificationSummary(valid_y, logit_reg.predict(valid_X[embeded_lr_feature]))

Confusion Matrix (Accuracy 0.9344)

       Prediction
Actual  0  1
     0  2  1
     1  3 55


## DecisionTree

In [86]:
embeded_dt_feature

['instkind_hospital',
 'instkind_traditional_clinic',
 'instkind_nan',
 'revenue1',
 'salescost1',
 'sga1',
 'salary1',
 'liquidAsset1',
 'quickAsset1',
 'receivableS1',
 'inventoryAsset2',
 'receivableL2',
 'employee2']

In [87]:
classTree = DecisionTreeClassifier(random_state=0)
classTree.fit(train_X_smote[embeded_dt_feature], train_y_smote)

DecisionTreeClassifier(random_state=0)

In [88]:
from dmba import classificationSummary

In [89]:
classificationSummary(train_y_smote, classTree.predict(train_X_smote[embeded_dt_feature]))

Confusion Matrix (Accuracy 0.9956)

       Prediction
Actual   0   1
     0 228   0
     1   2 226


In [90]:
classificationSummary(valid_y, classTree.predict(valid_X[embeded_dt_feature]))

Confusion Matrix (Accuracy 0.9344)

       Prediction
Actual  0  1
     0  0  3
     1  1 57


# to csv file  
(데이터프레임 이름).to_csv('csv파일 이름.csv', encoding="utf-8-sig" ) *한글 깨짐 현상 방지*

In [64]:
# train
Train

,OC,sido_choongchung,sido_gangwon,sido_gyeongin,sido_gyeongsang,sido_jeonra,sido_seoul,instkind_clinic,instkind_general_hospital,instkind_hospital,...,liquidLiabilities2,shortLoan2,NCLiabilities2,longLoan2,netAsset2,surplus2,employee1,employee2,ownerChange,bedCount_class
inst_id,,,,,,,,,,,,,,,,,,,,,
1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,19.222130,0.000000,20.099863,19.781657,21.686169,20.963246,4.143135,4.174387,0.0,3.0
3,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,18.841626,9.180719,18.812726,17.568559,18.838180,12.258481,6.687109,6.701960,0.0,4.0
4,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.459586,0.693147,0.0,4.0
7,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,23.557572,22.944579,23.755157,23.437961,23.284691,22.769775,6.498282,6.498282,0.0,4.0
9,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,24.126880,23.590810,23.784786,23.370010,15.531458,22.923325,5.332719,5.288267,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,21.685753,0.000000,22.641007,22.503290,22.405862,20.514569,5.267858,4.955827,0.0,3.0
423,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.382027,4.382027,0.0,3.2
426,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,21.498441,0.000000,22.158142,21.639056,21.904736,0.000000,3.860891,3.905642,0.0,2.6


In [68]:
Train.to_csv("C:/Users/kimch/Desktop/hospital/train.csv", encoding="utf-8-sig")

In [69]:
# test
Test

,OC,sido_choongchung,sido_gyeongin,sido_gyeongsang,sido_jeju,sido_jeonra,sido_seoul,instkind_clinic,instkind_general_hospital,instkind_hospital,...,liquidLiabilities2,shortLoan2,NCLiabilities2,longLoan2,netAsset2,surplus2,employee1,employee2,ownerChange,bedCount_class
inst_id,,,,,,,,,,,,,,,,,,,,,
2,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,24.648885,24.338050,22.275075,22.268698,23.501178,22.914314,6.542472,6.542472,0.0,3.0
5,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,24.463273,23.909618,23.950510,23.630167,22.055404,0.000000,5.940171,5.918894,0.0,2.0
6,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,24.047434,23.806220,0.000000,0.000000,0.000000,23.519657,5.285888,5.225619,0.2,3.0
8,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,22.980460,22.170624,21.662855,0.000000,24.390483,0.000000,6.634633,6.634633,0.0,4.0
10,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,22.742900,21.916712,24.318498,23.784183,23.971502,23.436150,6.082219,5.955837,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,23.279082,0.000000,21.735002,0.000000,23.351521,0.000000,6.329721,4.949613,0.0,3.0
425,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,21.042985,0.000000,21.310989,21.277151,22.872059,22.688514,4.890349,4.927254,0.0,3.2
429,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.588876,4.544446,0.0,3.0


In [71]:
Test.to_csv("C:/Users/kimch/Desktop/hospital/test.csv", encoding="utf-8-sig")

In [72]:
# train_featureselection_lr
train_lr = Train[['sido_choongchung', 'sido_gyeongsang', 'sido_jeonra', 'instkind_nursing_hospital', 'instkind_traditional_hospital',
 'revenue1', 'sga1', 'salary1', 'noe1', 'interest1', 'profit1', 'nonCAsset1', 'OnonCAsset1', 'debt1', 'surplus1', 'noi2',
 'noe2', 'interest2', 'profit2', 'inventoryAsset2', 'OnonCAsset2', 'shortLoan2', 'NCLiabilities2', 'longLoan2', 'surplus2', 'ownerChange',
 'bedCount_class', "OC"]]
train_lr

,sido_choongchung,sido_gyeongsang,sido_jeonra,instkind_nursing_hospital,instkind_traditional_hospital,revenue1,sga1,salary1,noe1,interest1,...,profit2,inventoryAsset2,OnonCAsset2,shortLoan2,NCLiabilities2,longLoan2,surplus2,ownerChange,bedCount_class,OC
inst_id,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,1.0,0.0,22.162515,22.099796,21.433189,16.539187,16.398469,...,19.152229,16.425505,18.798422,0.000000,20.099863,19.781657,20.963246,0.0,3.0,1
3,0.0,1.0,0.0,0.0,0.0,23.980280,23.481393,19.521179,16.823799,15.443340,...,17.025473,16.044684,15.372413,9.180719,18.812726,17.568559,12.258481,0.0,4.0,1
4,0.0,0.0,0.0,1.0,0.0,20.727778,19.918561,19.507228,10.308986,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,4.0,1
7,0.0,0.0,0.0,0.0,0.0,25.006954,24.981392,24.182293,20.954034,20.902985,...,20.546991,20.934483,22.088313,22.944579,23.755157,23.437961,22.769775,0.0,4.0,1
9,0.0,1.0,0.0,0.0,0.0,24.615974,24.587275,23.920337,21.073281,20.991191,...,17.301128,19.908536,20.486709,23.590810,23.784786,23.370010,22.923325,0.0,3.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,0.0,0.0,1.0,1.0,0.0,22.753346,22.498747,21.966804,20.427474,19.049695,...,18.287284,15.922774,18.157453,0.000000,22.641007,22.503290,20.514569,0.0,3.0,1
423,1.0,0.0,0.0,0.0,0.0,22.627929,0.000000,0.000000,18.756090,18.680716,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,3.2,0
426,0.0,1.0,0.0,0.0,0.0,0.000000,17.819096,15.830414,18.742626,18.742626,...,17.633437,15.389386,17.784997,0.000000,22.158142,21.639056,0.000000,0.0,2.6,0


In [73]:
train_lr.to_csv("C:/Users/kimch/Desktop/hospital/train_lr.csv", encoding="utf-8-sig")

In [74]:
# train_featureselection_dt
train_dt = Train[['instkind_hospital', 'instkind_traditional_clinic', 'instkind_nan', 'revenue1', 'salescost1', 'sga1', 'salary1',
 'liquidAsset1', 'quickAsset1', 'receivableS1', 'inventoryAsset2', 'receivableL2', 'employee2', "OC"]]
train_dt

,instkind_hospital,instkind_traditional_clinic,instkind_nan,revenue1,salescost1,sga1,salary1,liquidAsset1,quickAsset1,receivableS1,inventoryAsset2,receivableL2,employee2,OC
inst_id,,,,,,,,,,,,,,
1,0.0,0.0,0.0,22.162515,0.000000,22.099796,21.433189,20.735886,20.720935,19.968362,16.425505,0.000000,4.174387,1
3,0.0,0.0,0.0,23.980280,9.659782,23.481393,19.521179,19.074269,19.037270,10.917985,16.044684,0.000000,6.701960,1
4,0.0,0.0,0.0,20.727778,20.060616,19.918561,19.507228,19.422937,19.351593,15.906875,0.000000,0.000000,0.693147,1
7,0.0,0.0,0.0,25.006954,0.000000,24.981392,24.182293,23.291406,23.168630,0.000000,20.934483,0.000000,6.498282,1
9,0.0,0.0,0.0,24.615974,0.000000,24.587275,23.920337,22.566524,22.493677,22.134087,19.908536,0.000000,5.288267,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,0.0,0.0,0.0,22.753346,19.844626,22.498747,21.966804,21.744494,21.741939,21.309449,15.922774,0.000000,4.955827,1
423,1.0,0.0,0.0,22.627929,22.827400,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.382027,0
426,1.0,0.0,0.0,0.000000,0.000000,17.819096,15.830414,22.075806,22.075806,20.847528,15.389386,0.000000,3.905642,0


In [75]:
train_dt.to_csv("C:/Users/kimch/Desktop/hospital/train_dt.csv", encoding="utf-8-sig")